In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### ※ 파라미터 설정

In [ ]:
# 파라미터 설정
train_pct = 1 # 학습용 데이터 퍼센트
test_pct = 0 # 검증용 데이터 퍼센트
max_length = 1000 # 인코딩 입력 데이터 최대 길이
label_len = 325 # 디코딩 입력 데이터 최대 길이
batch_size = 4 

EPOCHS = 5
warmup_ratio = 0.1 # 학습 스케줄러 사용 시 warm_up 비율
learning_rate = 1e-4
max_grad_norm = 1

## 1. 패키지 로드 및 데이터 로드

### 1-1 필요 패키지 로드

In [ ]:
!pip install transformers
!pip install einops

In [ ]:
import transformers
import os
import re
import numpy as np
import requests
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.nn.utils as torch_utils
import time
from tqdm import tqdm

import json
from glob import glob

import warnings
warnings.filterwarnings(action='ignore')

from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from einops.layers.torch import Rearrange, Reduce

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score


# 토크나이저 설치
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0,1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)

Device: cuda


### 1-2 데이터 로드
- json 파일 로드 후 데이터를 데이터 프레임 형태로 변환

In [6]:
DIR = "/content/drive/MyDrive/LG_자연어/data"

In [7]:
# 데이터 로드 및 데이터 프레임 형태로 전환

TRAIN_SOURCE = os.path.join(DIR, "train.json")
TEST_SOURCE = os.path.join(DIR, "test.json")

with open(TRAIN_SOURCE) as f:
    TRAIN_DATA = json.loads(f.read())
    
with open(TEST_SOURCE) as f:
    TEST_DATA = json.loads(f.read())


# DataFrame 형태로 변환
train = pd.DataFrame(columns=['uid','agenda','title', 'context', 'summary'])

uid = 1000
for data in TRAIN_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '

        train.loc[uid, 'uid'] = uid
        train.loc[uid, 'title'] = data['title']
        train.loc[uid, 'agenda'] = agenda
        train.loc[uid, 'context'] = context[:-1]
        train.loc[uid, 'summary'] = data['label'][agenda]['summary']
        uid += 1

test = pd.DataFrame(columns=['uid','agenda', 'title', 'context'])
uid = 2000
for data in TEST_DATA:
    for agenda in data['context'].keys():
        context = ''
        
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        test.loc[uid, 'uid'] = uid
        test.loc[uid, 'title'] = data['title']
        test.loc[uid, 'agenda'] = agenda
        test.loc[uid, 'context'] = context[:-1]
        uid += 1
test['summary'] = '공백'

## 2. 전처리
- 특수문자 제거
- 문장 길이 측정 : 1) 인코더 및 디코더 입력 길이 판단 시 사용
- 특징 학습을 위한 레이블 설정 : 본문 길이, 요약문 길이, 요약문 끝문장의 형태를 대상으로 군집화를 수행하여 해당 군집 번호를 레이블로 사용 --> 해당 레이블을 함께 학습시키고 발생하는 손실값을 합하여 요약에 대한 학습과 문장 특징에 대한 학습을 동시에 진행 --> 특징을 고려하는 보다 안정적인 요약 모델을 생성하는 것이 가능

### 2-1 텍스트 전처리

In [8]:
# 전체 문서 기준, 높은 빈도수를 나타내는 문장을 필요없는 문장으로 판단하여 제거

text_list = []
for i in train['context']:
    for j in i.split('.')[:-1]:
        text_list.append(j)

text_value_counts = pd.DataFrame(pd.DataFrame(text_list)[0].value_counts()).reset_index()

remove_list = list(text_value_counts[text_value_counts[0]>=360]['index'])

def split_dot(df):
    return df.split('.')

train['context_split'] = train['context'].apply(split_dot)
test['context_split'] = test['context'].apply(split_dot)

train['context_split_removed'] = train['context_split'].apply(lambda x: [a for a in x if a not in remove_list])
test['context_split_removed'] = test['context_split'].apply(lambda x: [a for a in x if a not in remove_list])


train['context'] = train['context_split_removed'].apply(lambda x: '.'.join(x))
test['context'] = test['context_split_removed'].apply(lambda x: '.'.join(x))

train = train.drop(['context_split','context_split_removed'],axis=1)
test = test.drop(['context_split','context_split_removed'],axis=1)

train['total'] = train.title + ' <t> ' + train.context
test['total'] = test.title + ' <t> ' + test.context

In [9]:
in_df = train.copy()
in_df.head()

,uid,agenda,title,context,summary,total
1000,1000,AGENDA_1,제207회 완주군의회(임시회) 제 1 차 본회의회의록,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 <t> 의석을 정돈하여 주시기...
1001,1001,AGENDA_2,제207회 완주군의회(임시회) 제 1 차 본회의회의록,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 <t> 의사팀장 수고하셨습니다...
1002,1002,AGENDA_3,제207회 완주군의회(임시회) 제 1 차 본회의회의록,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 <t> 다음은 의사일정 제2항...
1003,1003,AGENDA_4,제207회 완주군의회(임시회) 제 1 차 본회의회의록,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...,제207회 완주군의회(임시회) 제 1 차 본회의회의록 <t> 다음은 의사일정 제3항...
1004,1004,AGENDA_1,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록 <t> 의석을 정...


In [10]:
# 특수문자 제거 함수
def clean_text(text):
    text = re.sub("\(", ' ', text)
    text = re.sub("\)", ' ', text)
    text = re.sub("[『 』]", ' ', text)
    return text

in_df["total_clean"] = in_df["total"].apply(clean_text)
in_df["summary_clean"] = in_df["summary"].apply(clean_text)
test["total_clean"] = test["total"].apply(clean_text)

In [11]:
# 문장의 처음과 끝에 사인을 삽입
def sos_eos(df):
    return '<s> '+df+' </s>'

in_df['total_clean'] = in_df['total_clean'].apply(sos_eos)
in_df['summary_clean'] = in_df['summary_clean'].apply(sos_eos)

test['total_clean'] = test['total_clean'].apply(sos_eos)
test['summary'] = test['summary'].apply(sos_eos)

### 2-2 군집화를 활용한 문장 특징 레이블 생성

In [12]:
# 문장 길이 출력 함수 
def cal_len(text):
    return len(text.split())

in_df['context_len'] = in_df['total_clean'].apply(cal_len)
in_df['target_len'] = in_df['summary_clean'].apply(cal_len)
in_df['target_len_total'] = in_df['summary'].apply(lambda x: len(x))
test['len'] = test['total_clean'].apply(cal_len)

# Agenda 레이블 인코딩
last_sumtext = list(train["agenda"].unique())
in_df["agenda_label"] = train["agenda"].apply(lambda x: last_sumtext.index(x))

In [13]:
k = StandardScaler().fit_transform(in_df[['context_len','target_len_total','agenda_label']])

kmeans = KMeans(n_clusters=9,random_state=0)
labels = kmeans.fit_predict(k)
in_df['label'] = labels

print('실루엣 스코어 : {0:.3f}'.format(silhouette_score(k,labels)))

실루엣 스코어 : 0.408


### 2-3 학습 데이터 준비

In [14]:
class Summary_dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, lasttext=None):
        self.encodings = encodings["input_ids"]
        self.token_type = encodings["token_type_ids"]
        self.masking = encodings["attention_mask"]
        self.labels = labels
        self.lasttext = lasttext            
            
    def __getitem__(self, idx):
        item = {}
        if self.encodings[idx][-1]!=3:
            self.encodings[idx][-1] = 1 
            
        if self.labels['input_ids'][idx][-1] !=3:
            self.labels['input_ids'][idx][-1] = 1
        
        if self.lasttext is not None:
            item['last_text'] = torch.tensor(self.lasttext.iloc[idx])
        
        item['input_ids'] = torch.tensor(self.encodings[idx])
        item['attention_mask'] = torch.tensor(self.masking[idx])
        item['token_type_ids'] = torch.tensor(self.token_type[idx])
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])  # torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings)

In [15]:
## 학습 및 검증 세트 설정
in_df = in_df.sample(len(in_df), random_state=20)
train_sub = int(len(in_df) * train_pct)
# test_sub = int(len(in_df) * test_pct) + train_sub

train_df = in_df[0:train_sub]
test_df = in_df[train_sub:]
# val_df = in_df[test_sub:]

train_texts = list(train_df['total_clean'])
test_texts = list(test_df['total_clean'])
# val_texts = list(val_df['text'])

train_decode = list(train_df['summary_clean'])
test_decode = list(test_df['summary_clean'])
# val_decode = list(val_df['summary'])

test_df_texts = list(test['total_clean'])
test_df_decode = list(test['summary'])


## 토크나이징
train_encodings = tokenizer(train_texts,max_length=max_length, truncation=True, padding='max_length',add_special_tokens = True,return_tensors="pt")
# test_encodings = tokenizer(test_texts,max_length=max_length, truncation=True, padding=True)
train_labels = tokenizer(train_decode,max_length=label_len, truncation=True, padding='max_length',add_special_tokens = True,return_tensors="pt")
# test_labels = tokenizer(test_decode,max_length=50, truncation=True, padding=True)

test_df_encodings = tokenizer(test_df_texts,max_length=max_length, truncation=True, padding='max_length',add_special_tokens = True,return_tensors="pt")
test_df_labels = tokenizer(test_df_decode,max_length=label_len, truncation=True, padding=True)

In [16]:
train_dataset = Summary_dataset(train_encodings, train_labels, lasttext=in_df["label"])
# test_dataset = Summary_dataset(test_encodings, test_labels)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=5, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1)

## 3. 모델 설계
#### 모델 특징
- 한 모델에서 요약에 대한 학습과 문장 특징 학습(군집화를 통해 구한 클래스에 대한 분류 학습)을 동시에 진행
- 각 학습을 통해 구해진 loss를 A + (B/10) 비율로 합산하고 이를 역전파를 통해 업데이트
- 문장의 특징(context의 길이, 요약문의 길이, 안건)을 함께 학습하기 때문에 출력문의 길이와 형태를 보다 잘 나타내어 ***안정적인 학습***이 가능 

#### 세부 조정 사항
- 주어진 학습데이터가 상대적으로 적기 때문에 인코딩 층, 디코딩 층 각각의 최하단 부분과 출력층까지만 가중치 업데이트 진행
- 2 epoch까지 요약문과 문장 특징을 함께 학습하고, 이후 3번의 epoch 요약문 학습만 진행
- 학습률 스케줄을 적용하여 처음에는 1e-4로 학습 후 warm ratio 초과시 get_cosine_schedule_with_warmup을 이용하여 학습률 조정


In [19]:
class BART_Classification(nn.Module):
    def __init__(self,n_classes, emb_size=768):
        super().__init__()
        self.bart_model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')
        self.reduce = Reduce('b n e -> b e', reduction='mean')
        self.norm = nn.LayerNorm(n_classes)
        self.linear = nn.Linear(emb_size, n_classes)
        self.softmax=nn.LogSoftmax(dim=-1)
        self.relu = nn.ReLU()
        
    def forward(self,input_,mask_,intrg):
        out = self.bart_model.forward(input_,mask_,intrg)
        hidden_state = out.encoder_last_hidden_state
                
        x = self.reduce(hidden_state)
        x = self.linear(x)
        x = self.norm(x)

        x = self.softmax(x)
        return x, out

model = BART_Classification(in_df["label"].nunique())
model = model.to(device)

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

### 3-1 모델 세부 조정

##### - KoBART 모델 층 구조
- encoding part : 2 ~ 97 / Layer norm 98 ~ 99
- decoding part : Embedding, PE  100 ~ 101 / Decoding 층 102 ~ 257 / Layer norm 258~259

- 총 개수 259 개

##### - 주어진 학습 데이터가 상대적으로 적기 때문에 인코딩 층, 디코딩 층 각각의 최하단 부분과 출력층까지 가중치 업데이트 진행
##### - 초기에 1e-4로 학습 후 warm ratio 초과시 get_cosine_schedule_with_warmup을 이용하여 학습률 조정

In [20]:
n=0
for name, child in model.named_children():
    if n==0:
      h=0
      for param in child.parameters():
        if (95<=h) or (98<=h<=255):
            param.requires_grad = False
        h+=1
    n+=1

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
# optimizer =optim.SGD(model.parameters(), lr=10e-5, momentum=0.9)
t_total = len(train_dataloader) * EPOCHS
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

criterion = nn.CrossEntropyLoss()

### 3-2 학습

In [21]:
torch.cuda.empty_cache() # 캐시 삭제

for Epoch in range(EPOCHS): 
    train_loss = []
    first_time = time.time()
    epoch_start = time.time()
    model.train()
    torch.set_grad_enabled(True)
    for batch_idx, data in enumerate(train_dataloader):
        optimizer.zero_grad()
       
        input_ = data['input_ids'].long().to(device)
        mask_ = data['attention_mask'].long().to(device)
        trg_ = data["labels"]
        intrg = torch.tensor(trg_[:,:-1]).to(device)
        outtrg = torch.tensor(trg_[:,1:]).to(device)
        
        c_out, out = model.forward(input_,mask_,intrg)
#         c_out = c_bart.forward(input_,mask_,intrg)
        
        _,_,vocab = out[0].size()
        out_ = out[0].view(-1,vocab)
        outtrg = outtrg.view(-1)

        c_loss = criterion(c_out.cpu(),data["last_text"].long().cpu())
        

        # 2 에폭까지는 함께 학습, 이후 3 에폭은 요약문 대상 학습만 진행
        # pad 부분에 대해 손실을 구하지 않도록 설정
        if Epoch<2:
            loss_ = criterion(out_,outtrg)
            nopad = torch.logical_not(torch.eq(outtrg, 3))
            nopad = torch.tensor(nopad, dtype=loss_.dtype)
            loss_ = nopad * loss_
        
            loss = torch.sum(loss_)/torch.sum(nopad) + c_loss/10
        else: 
            loss_ = criterion(out_,outtrg)
            nopad = torch.logical_not(torch.eq(outtrg, 3))
            nopad = torch.tensor(nopad, dtype=loss_.dtype)
            loss_ = nopad * loss_
        
            loss = torch.sum(loss_)/torch.sum(nopad)


        loss.backward()
        torch_utils.clip_grad_norm_(model.parameters(),1)
        optimizer.step()
        
        scheduler.step()

        if batch_idx%20==0:
            second_time = time.time()
            print("Train Epoch: {} [{}/{}({:.0f}%)]\tTrain Loss: {:.6f}\t runtime={:.1f}sec".format(Epoch, batch_idx * batch_size, train_sub,100.*batch_idx* batch_size/train_sub, loss.item(),(second_time-first_time)))
            first_time = second_time
            
        del input_,mask_,trg_, out, outtrg, loss
        
        torch.cuda.empty_cache()
        
    epoch_end = time.time()

    print("Train Epoch Takes {:.2f}min".format((epoch_end - epoch_start)/60))

Train Epoch: 0 [0/2994(0%)]	Train Loss: 12.026088	 runtime=0.9sec
Train Epoch: 0 [80/2994(3%)]	Train Loss: 11.242523	 runtime=12.2sec
Train Epoch: 0 [160/2994(5%)]	Train Loss: 9.296627	 runtime=12.2sec
Train Epoch: 0 [240/2994(8%)]	Train Loss: 7.047321	 runtime=12.2sec
Train Epoch: 0 [320/2994(11%)]	Train Loss: 4.654191	 runtime=12.2sec
Train Epoch: 0 [400/2994(13%)]	Train Loss: 2.787152	 runtime=12.2sec
Train Epoch: 0 [480/2994(16%)]	Train Loss: 1.613371	 runtime=12.3sec
Train Epoch: 0 [560/2994(19%)]	Train Loss: 0.697777	 runtime=12.2sec
Train Epoch: 0 [640/2994(21%)]	Train Loss: 0.349504	 runtime=12.2sec
Train Epoch: 0 [720/2994(24%)]	Train Loss: 0.237136	 runtime=12.2sec
Train Epoch: 0 [800/2994(27%)]	Train Loss: 0.451012	 runtime=12.2sec
Train Epoch: 0 [880/2994(29%)]	Train Loss: 0.177615	 runtime=12.2sec
Train Epoch: 0 [960/2994(32%)]	Train Loss: 0.324094	 runtime=12.2sec
Train Epoch: 0 [1040/2994(35%)]	Train Loss: 0.212403	 runtime=12.2sec
Train Epoch: 0 [1120/2994(37%)]	Train L

In [22]:
torch.cuda.empty_cache()

## 4. 추론

In [23]:
sub = pd.read_csv("/content/drive/MyDrive/LG_자연어/data/sample_submission.csv")

In [24]:
test_df_dataset = Summary_dataset(test_df_encodings, train_labels)
test_df_dataloader = DataLoader(test_df_dataset, batch_size=2, num_workers=5)

In [27]:
def clean_text_2(text):
    text = text.split("</s>")[0]
    text = text.replace("<s>"," ")
    text = text.replace("<usr>"," ")
    text = text.replace("<pad>"," ")
    test = re.sub('<[a-zA-Z]+>'," ",text)
    text = text.replace("▁"," ")
    text = re.sub("\n", ' ', text)
    text = re.sub("\xa0", ' ', text)   
    text = re.sub('<',' ',text)
    text = re.sub('>',' ',text)
    text = re.sub('\s+', ' ', text)

    return text

### 4-1 학습된 모델을 이용한 추론

In [28]:
test_loss = 0
correct = 0

test_sum = []
model.eval()
for data in tqdm(test_df_dataloader):
    
    input_ = data['input_ids'].to(device)    
    out = model.bart_model.generate(input_, max_length=500)
    torch.cuda.empty_cache()
    
    for i in range(len(out)):
        text = tokenizer.convert_ids_to_tokens(out[i])
        text = "".join(text)
        text = clean_text_2(text)        
        test_sum.append(text)


sub.iloc[:,1] = test_sum

100%|██████████| 253/253 [02:17<00:00,  1.83it/s]


### 4-2 후처리
- 특수문자 제거
- 앞 뒤 공백 제거

In [29]:
sub["summary"] = sub["summary"].apply(clean_text_2)

for i in range(len(sub['summary'])):
    if sub['summary'][i][0] == ' ':
        sub.loc[i,'summary'] = sub['summary'][i][1:]
    if sub['summary'][i][-1] == ' ':
        sub.loc[i,'summary'] = sub['summary'][i][:-1]

### 4-3 출력 요약문 확인

In [34]:
# 출력 요약문 확인
cnt = 0
for i in sub['summary']:
    print('%d번 요약문'%cnt)
    print(i)
    print()
    cnt +=1
    if cnt == 10:
        break

0번 요약문
음성군의회 제235회 제1차 정례회 제1차 본회의 개의 선포.

1번 요약문
음성군의회 제235회 제1차 정례회 회기는 2012년 6월 21일부터 6월 28일까지 8일간으로 가결됨.

2번 요약문
제235회 제1차 정례회 회의록 서명의원으로 조천희 의원, 손달섭 의원이 선출됨.

3번 요약문
예산결산특별위원회 위원은 손수종 의원, 이한철 의원, 남궁유 의원, 조천희 의원, 손달섭 의원, 이대웅 의원, 김순옥 의원으로 구성함. 특별위원회는 6월 25일 하루동안 2011년도 예비비 지출 승인안, 2011회계 세입 세출 결산 승인안, 2011년도 회계 기금운용 성과분석보고 등을 회부하여 의사일정에 따라 심사하고자 구성함. 해당 안건은 가결됨.

4번 요약문
음성군 환경 분야의 현지확인 특별위원회 위원은 손수종 의원, 이한철 의원, 남궁유 의원, 조천희 의원, 손달섭 의원, 이대웅 의원, 김순옥 의원으로 구성함. 현지 확인 기간은 6월 27일부터 6월 28일까지 이틀간이며, 현지 확인 결과 결과보고는 제237회 임시회 본회의 시로 가결됨.

5번 요약문
주요사업 현지확인 결과보고서가 채택됨.

6번 요약문
제322회 음성군의회 임시회 제1차 본회의 개의 선포.

7번 요약문
제322회 음성군의회 임시회 회기는 4월 22일부터 4월 27일까지 6일간으로 가결됨.

8번 요약문
제322회 임시회 회의록 서명의원으로 김영섭 부의장, 김영호 의원이 선출됨.

9번 요약문
2020년도 제1회 세입 세출 추가경정예산안 및 기금운용계획 변경안 제안설명.



### 4-4 결과 저장

In [35]:
sub.to_csv("/content/drive/MyDrive/LG_자연어/submission/submission_f.csv", index=False, encoding = 'utf-8')